### 案例分析：客户精准营销(ML模型)
>机器学习(Machine Learning, ML)是一门多领域交叉学科，涉及概率论、统计学、逼近论、凸分析、算法复杂度理论等多门学科。本案例采用领先的机器学习技术，研究UCI机器学习库中的「银行营销数据集(Bank Marketing Data Set)」，分析每一个客户的个性化需求，实现精准的产品营销与投放。


一、背景与目标 

1.1 背景

根据客户历史营销响应数据，结合对市场未来需求数据、相关行业政策数据等，预测未来周期内客户营销响应，用以指导业务人员有针对性的营销，提高工作效率与经济效益。


1.2 目标

本实例使用的这些数据与葡萄牙银行机构的营销活动相关。这些营销活动以电话为基础，一般，银行的客服人员需要联系客户至少一次，以此确认客户是否将认购该银行的产品（定期存款）。因此，与该数据集对应的任务是「分类任务」，在本实例中，主要希望实现以下目标：

- **通过二分类算法，预测客户是否购买该款产品。**

| NO | 字段名称 | 数据类型 | 字段描述 |
| :-----| ----: | :----: | :----: |
| 1 | ID | Int | 客户唯一标识 |
| 2 | age | Int | 客户年龄 |
| 3 | job | String | 客户的职业 |
| 4 | marital | String | 婚姻状况 |
| 5 | education | String | 受教育水平 |
| 6 | default | String | 是否有违约记录 |
| 7 | balance | Int | 每年账户的平均余额 |
| 8 | housing | String | 是否有住房贷款 |
| 9 | loan | String | 是否有个人贷款 |
| 10 | contact | String | 与客户联系的沟通方式 |
| 11 | day | Int | 最后一次联系的时间（几号） |
| 12 | month | String | 最后一次联系的时间（月份） |
| 13 | duration | Int | 最后一次联系的交流时长 |
| 14 | campaign | Int | 在本次活动中，与该客户交流过的次数 |
| 15 | pdays | Int | 距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过） |
| 16 | previous | Int | 在本次活动之前，与该客户交流过的次数 |
| 17 | poutcome | String | 上一次活动的结果 |
| 18 | y | Int | 预测客户是否会订购定期存款业务

In [74]:
import pandas as pd
import os
os.getcwd()
os.chdir("C:\\Users\\ShangFR\\Desktop\\eBrain\\Bank_demo")

In [75]:
data = pd.read_csv('bankdata\\train_set.csv', encoding='utf-8')  # 导入原始数据,指定UTF-8编码
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [76]:
#查看数据概述
explore = data.describe(include=None).T 
explore['null'] = len(data) - explore['count']  # 计算空值数
print(explore) 

            count          mean          std     min     25%      50%  \
ID        25317.0  12659.000000  7308.532719     1.0  6330.0  12659.0   
age       25317.0     40.935379    10.634289    18.0    33.0     39.0   
balance   25317.0   1357.555082  2999.822811 -8019.0    73.0    448.0   
day       25317.0     15.835289     8.319480     1.0     8.0     16.0   
duration  25317.0    257.732393   256.975151     0.0   103.0    181.0   
campaign  25317.0      2.772050     3.136097     1.0     1.0      2.0   
pdays     25317.0     40.248766   100.213541    -1.0    -1.0     -1.0   
previous  25317.0      0.591737     2.568313     0.0     0.0      0.0   
y         25317.0      0.116957     0.321375     0.0     0.0      0.0   

              75%       max  null  
ID        18988.0   25317.0   0.0  
age          48.0      95.0   0.0  
balance    1435.0  102127.0   0.0  
day          21.0      31.0   0.0  
duration    317.0    3881.0   0.0  
campaign      3.0      55.0   0.0  
pdays        -1.0

In [77]:
print(data.y.value_counts())
data.shape
data_s = data.select_dtypes(include=['object'])

for i in range(len(data_s.columns)):
    u = sum(data_s.iloc[:,i] == 'unknown')
    if u > 0:
        print(data_s.columns[i],'缺失率:%.2f%%'%(100*u/36169))
    else:
        pass
    
# data_df = data.drop(['ID','contact','poutcome'], axis=1)
# data_df.drop_duplicates(subset=['A','B'],keep='first',inplace=True)



0    22356
1     2961
Name: y, dtype: int64
job 缺失率:0.45%
education 缺失率:2.94%
contact 缺失率:20.13%
poutcome 缺失率:57.17%


In [78]:
# ont-hot 编码
data_df = data.drop(['ID'], axis=1)
print(data_df.dtypes)
print(data_df.dtypes.value_counts())
data_dummy = pd.get_dummies(data_df.select_dtypes(include=['object']))
data_oh = pd.concat([data_dummy, data_df.select_dtypes(exclude=['object'])], axis=1)


age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y             int64
dtype: object
object    9
int64     8
dtype: int64


In [79]:
# 数据集划分
from sklearn.model_selection import train_test_split

X = data_oh.drop(['y'], axis=1)   
y = data_oh.y                  
#测试集占训练集30%
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=90)


GBDT(Gradient Boosting Decision Tree)是目前工业和各种竞赛中非常抢手的模型，性能表现出色，特别是XgBoost，LightGBM推出后，模型性能和运行效率进一步提升。GBDT模型是一个集成模型，基分类器采用CART，集成方式为Gradient Boosting。

CART是一个分类回归二叉决策树，构建一棵二叉树，主要涉及到一下一个问题：

怎么分裂一个特征？
怎么选择最佳分裂特征？
确定分裂的停止条件？
决策树的优化：剪枝方法？
因为CART是一棵二叉树，所以在分裂特征时与 ID3、C4.5有区别。
CART在分类时采用最小平方误差来选择最优切分特征和切分点。

Boosting
Boosting是一种模型的组合方式，我们熟悉的AdaBoost就是一种Boosting的组合方式。和随机森林并行训练不同的决策树最后组合所有树的bagging方式不同，Boosting是一种递进的组合方式，每一个新的分类器都在前一个分类器的预测结果上改进，所以说boosting是减少bias而bagging是减少variance的模型组合方式。
下面是GDBT的一个简单例子：判断用户是否会喜欢电脑游戏，特征有年龄，性别和职业。需要注意的是，GBDT无论是用于分类和回归，采用的都是回归树，分类问题最终是将拟合值转换为概率来进行分类的。

![graph.png](attachment:graph.png)
在上图中,每个用户的最后的拟合值为两棵树的结果相加。

GBDT的主要优点：

　　1）可以灵活的处理各种类型的数据

　　2）预测的准确率高

　　3）使用了一些健壮的损失函数，如huber，可以很好的处理异常值

GBDT的缺点：

　　1）由于基学习器之间的依赖关系，难以并行化处理，不过可以通过子采样的SGBT来实现部分并行。

 

In [80]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

GBDT= GradientBoostingClassifier(random_state=90)
GBDT.fit(Xtrain, ytrain)
GBDT.score(Xtest, ytest)
y_pred= GBDT.predict(Xtest)
y_predprob= GBDT.predict_proba(Xtest)[:,1]
print("Accuracy : %.4g" % metrics.accuracy_score(ytest.values, y_pred))
print("AUC Score (Train): %f" % metrics.roc_auc_score(ytest, y_predprob))



Accuracy : 0.9072
AUC Score (Train): 0.922007


机器学习中的一大难点就是参数调优，每个模型会有很多可调节的模型参数。
模型参数（Hyperparameters）调优的通用做法是Grid Search或者Random Search，Sklearn中已经提供了相应的方法GridSearchCV和RandomizedSearchCV。 这两种搜索方法都包括Search和CV两步，即搜索和交叉验证。

GridSearch在需要调节参数的指定取值范围，遍历所有组合寻，比如有两个参数，第一个有两种值，第二个有三种，那就会有六种组合。GridSearch会对每种参数组合做一遍交叉验证，记录模型得分，最后找到得分最好的那个参数组合。GridSearchCV可以保证在指定的参数范围内找到精度最高的参数，但它要求遍历所有可能参数的组合，在面对大数据集和多参数的情况下会非常耗时。

RandomizedSearchCV的使用方法其实是和GridSearchCV一致的，但它以随机在参数空间中采样的方式代替了GridSearchCV对于参数的网格搜索，在对于有连续变量的参数时，RandomizedSearchCV会将其当作一个分布进行采样，它的搜索能力取决于设定的循环n_iter参数。

In [81]:
from sklearn.model_selection import GridSearchCV
import numpy as np
# 网格寻参,寻找最优'max_depth'、'min_samples_split'；耗时费力，数据集大的时候不好用

param_grid = {'max_depth':np.arange(10, 20, 5), #决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split
              'min_samples_split':np.arange(50, 100, 10)}
print(" param_grid: ", param_grid)

gbdt = GradientBoostingClassifier(random_state=90)
GS = GridSearchCV(gbdt,param_grid,n_jobs=-1,cv=3)
GS.fit(Xtrain,ytrain)
# print('网格搜索-度量记录：',GS.cv_results_)  # 包含每次训练的相关信息
print('网格搜索-最佳度量值:',GS.best_score_)  # 获取最佳度量值
print('网格搜索-最佳参数：',GS.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
print('网格搜索-最佳模型：',GS.best_estimator_)  # 获取最佳度量时的分类器模型


 param_grid:  {'max_depth': array([10, 15]), 'min_samples_split': array([50, 60, 70, 80, 90])}


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
                                                  presort=

In [83]:
from sklearn.model_selection import RandomizedSearchCV

# 寻参
param_dist = {
    'learning_rate':np.linspace(0.1,2,20), #步长(learning rate)和迭代次数(n_estimators)
    'n_estimators':range(80,200,4),
    'max_depth':range(2,15,1), #决策树最大深度max_depth和内部节点再划分所需最小样本数min_samples_split、叶子节点最少样本数
    'min_samples_split':range(100,801,200),
    'min_samples_leaf':range(60,101,10),
    'max_features':range(7,20,2), #最大特征数max_features
    'subsample':np.linspace(0.7,0.9,20) #子采样的比例
}

gbdt = GradientBoostingClassifier()
RS = RandomizedSearchCV(gbdt,param_dist,cv = 3,scoring = 'accuracy',n_iter=10,n_jobs = -1)

#在训练集上训练
RS.fit(Xtrain, ytrain)
#返回最优的训练器

#print('网格搜索-度量记录：',RS.cv_results_)  # 包含每次训练的相关信息
print('网格搜索-最佳度量值:',RS.best_score_)  # 获取最佳度量值
print('网格搜索-最佳参数：',RS.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
print('网格搜索-最佳模型：',RS.best_estimator_)  # 获取最佳度量时的分类器模型


网格搜索-最佳度量值: 0.9050279329608939
网格搜索-最佳参数： {'subsample': 0.731578947368421, 'n_estimators': 148, 'min_samples_split': 700, 'min_samples_leaf': 60, 'max_features': 9, 'max_depth': 2, 'learning_rate': 0.1}
网格搜索-最佳模型： GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=2,
                           max_features=9, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=60, min_samples_split=700,
                           min_weight_fraction_leaf=0.0, n_estimators=148,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=0.731578947368421,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [84]:
# 建模
GBDT = RS.best_estimator_
GBDT.fit(Xtrain, ytrain)
GBDT.score(Xtest, ytest)
y_pred= GBDT.predict(Xtest)
y_predprob= GBDT.predict_proba(Xtest)[:,1]
print("Accuracy : %.4g" % metrics.accuracy_score(ytest.values, y_pred))
print("AUC Score (Train): %f" % metrics.roc_auc_score(ytest, y_predprob))



Accuracy : 0.9051
AUC Score (Train): 0.916130


In [85]:
# 模型保存与载入
import joblib
joblib.dump(GBDT, "model\\GBDT.model")
GBDT = joblib.load("model\\GBDT.model")

In [93]:
# 载入测试集，结果输出

data_test = pd.read_csv('bankdata\\test_set.csv')
ID = data_test.ID
data_test.drop(['ID'], axis=1, inplace=True)
data_test_dummy = pd.get_dummies(data_test.select_dtypes(include=['object']))
data_test_oh = pd.concat([data_test_dummy, data_test.select_dtypes(exclude=['object'])], axis=1)

pred = GBDT.predict_proba(data_test_oh)
data_out = pd.DataFrame(pred, index=ID, columns=['pred0', 'pred'])
data_out.drop('pred0', axis=1, inplace=True)
data_out.to_csv('bankdata\\result1009.csv')